# Laboratory 9 - Sample Report

## Water Surface Profiles: Hydraulic Jump


## Purpose

Examine behavior in open channels, in particular determining Manning's n for a portion of a channel, and cerating and observing the alternate and sequent depths in a hydraulic jump.

## Objective(s)
1. Create a hydraulic jump, stabilizing it, then measuring the alternate and sequent depths and comparing these to calculated values based on the discharge.

## Apparatus and Methods

## Laboratory Apparatus

The apparatus is a recirculating water flume (photo below), width 1 ft, comprising a supply tank (in the flume base) a head tank, two pumps, rectangular channel with side rails, depth gauges, total head tubes, bed tappings and downstream control gate.

![](flumephoto.png)

The experiment is comprised of: 
1. measuring depth over a low height weir and determine discharge in the flume
2. creating a hydraulic jump, stabilizing it (my varying slope and slucie gate opening), then measuring the alternate and sequent depths and comparing these to calculated values based on the discharge.

### Experimental Procedure

**1. Zero the Depth Gauge**

1. Position the depth gauge directly over the weir.  
2. Move the transit (the vertical sliding carriage) to approximately the middle of its motion range.  
3. Lower the point gauge until the tip just touches the top of the weir.  
4. Tighten the adjustment screw gently to lock the pointer in place.  
5. Move the gauge slightly upstream and verify that it can still reach the channel bottom.  
6. The gauge is now zeroed — all subsequent readings will be relative to the weir crest.

**2. Establish and Measure the Hydraulic Jump**

1. Ensure the flume slope is set to approximately 4% (as pre-set by the instructor or TA).  
   This slope may appear steep, but it produces a well-defined jump within the short flume.  
2. Start both recirculating pumps. The valves should already be fully open; verify with the instructor.  
3. Raise the sluice gate to about 50 mm above the channel bottom to establish flow.  
   - Monitor the head tank — if it fills too quickly or the level rises above the rail, raise the gate slightly higher.  
   - The target condition is steady flow with the head tank level just below the rail height.  
4. Adjust the sluice gate opening and/or channel slope until a visible hydraulic jump forms roughly midway between the head tank and the weir.  
   - The exact position is not critical, but it should be far enough downstream from the manometers to allow easy use of the depth gauge.  
5. Allow the flow to reach steady conditions.  
6. Measure the upstream (alternate) depth approximately halfway between the headgate and the jump.  
7. Measure the downstream (sequent) depth at a location where the surface appears relatively stable—typically about halfway between the jump and the weir.



## Measurements

The photograph below is an image showing the stable hydraulic jump

![](IMG_4076.jpg)

The approach depth was measure upstream at station -330 cm (to the left of the image area).  The sequent depth was measured at station -210 cm (just to the right of the image area).

The photograph below is an image near the weir end of the experiment showing the flow over the weir

![](IMG_4077.jpg)

The flow depth at the weir (for discharge) is measure in the relatively smooth area upstream of the weir.


The table below lists the experimental measurements

|Station|Depth-to-Water (mm)|Weir Height (mm)|Flow Depth (mm)|$Q_{weir}$ cms|
|:---|---:|---:|---:|---:|
|Weir (0)|80.0|180.0|260.0|0.019|
|Alternate (-330)|-160.0|180.0| 20.0|N/A|
|Sequent (-210)| 10.0|180.0|190.0|N/A|

Note: Discharge is computed below and entered above

## Data Reduction and Analysis

### Discharge
The flow depth at the weir is used in the weir equation to determine discharge.

$
Q = C_d \, L \, \sqrt{2g} \, h^{3/2}
$

The equation is supplied with SI units to compute discharge in $m^3/s$.  The default $C_d$ = 0.611 is used.

In [24]:
# Discharge_from_weir (SI units)
def Q_weir(weir_width,weir_height,flow_depth,C_d = 0.611):
    import math
    h = flow_depth - weir_height
    g = 9.81
    Q_weir = C_d * weir_width * math.sqrt(2.0*g) * h**(3/2)
    return(Q_weir)

q_cms = Q_weir(310/1000,180/1000,260/1000)
print(f"Discharge: {q_cms:.3f} cubic meters per second")

Discharge: 0.019 cubic meters per second


### Alternate and Sequent Depths

To compute anticipated sequent depth the flow velocity is required in order to compute the Froude number.

In [25]:
# Velocity_from_Q_and_Depth
def V_r(channel_width,flow_depth,discharge):
    V_r = discharge/(channel_width*flow_depth)
    return(V_r)

print(f"Approach velocity: {V_r(310/1000,20/1000,0.019):.3f} meters per second")

Approach velocity: 3.065 meters per second


The Froude number is computed using the definition of Froude number for a rectangular channel

$Fr = \frac{V}{\sqrt{gD}}$

where $V$ is the flow velocity, and $D$ is the flow depth.

In [26]:
#Froude number (SI units)
def Fr(velocity,depth):
    import math
    g=9.81
    Fr=velocity/math.sqrt(g*depth)
    return(Fr)

print(f"Froude number {Fr(3.065,20/1000):.3f} ")

Froude number 6.920 


Now we can compute anticipated sequent depth (assuming the approach depth is correct)

In [27]:
#Conjugate depths
def yTwo(yOne,FrOne):
    import math
    yTwo=(yOne/2.0)*(math.sqrt(1.0+8.0*FrOne**2)-1.0)
    return(yTwo)

print(f"Sequent depth: {yTwo(20.0/1000,6.920):.3f} meters")

Sequent depth: 0.186 meters


The observed sequent depth was 0.190 meters, so the results agree nicely with theory (the error section below goes into more meaningful details).  Some additional analysis follows.

### Energy Loss

Hydraulic jumps are good energy disippators we can compute the loss in the jump using

In [28]:
#Head Loss
def NRG_loss(y1,y2):
    NRG_loss=((y2-y1)**3)/(4.0*y1*y2)
    return(NRG_loss)

print(f"Energy Loss {NRG_loss(20/1000,190/1000):.3f} meters") 

Energy Loss 0.323 meters


Probably more useful to express in terms of power

In [29]:
def power_loss(Q, hL, rho=1000.0, g=9.81):
    """Compute power dissipated (W) from head loss and flow rate."""
    gamma = rho * g  # specific weight of water (N/m³)
    P = gamma * Q * hL
    return P

print(f"Power loss: {power_loss(0.019,0.323):.3f} Watts")

Power loss: 60.204 Watts


### Jump Length

[Hager, W. H. (1992). Energy Dissipators and Hydraulic Jump. Kluwer Academic Publishers, Dordrecht, The Netherlands.](https://openlibrary.org/books/OL1557072M/Energy_dissipators_and_hydraulic_jump) presents empirical relationships for hydraulic jump characteristics derived from systematic flume experiments. The jump length correlation

$L=220~y_1~tanh[(Fr_1 -1)/22]$

is valid for Froude numbers between 2 and 15 and provides a practical estimate of jump length measured from the toe to where surface oscillations dissipate. Hager’s formulation is widely cited in design manuals and research on stilling basins and energy dissipators.

In the experiments the observed distance (Sequent station - Alternate station position) is 1.2 meters. Which is quite close to the predicted length below.

In [30]:
#Jump Length 
def jump_L(yOne,FrOne):
    import math
    jump_L=220*yOne*math.tanh((FrOne -1.0)/22.0)
    return(jump_L)

print(f"Jump Length : {jump_L(0.020,6.920):.3f} meters")

Jump Length : 1.156 meters


### Jump Efficiency

You can also estimate what percentage of the original energy is lost during a hydraulic jump (jump efficiency) This value is dependent mostly on the Froude number of the upstream flow and is governed by the following equation:

$\eta = 100\times\frac{(8Fr_1^2 +1)^{3/2} -4Fr_1^2+1}{8Fr_1^2(2+Fr_1^2)}$



In [31]:
#Jump Efficiency
def jump_Eff(FrOne):
    Frsq = FrOne**2
    jump_Eff = ((8.*Frsq+1.)**(3./2.)-4.*Frsq+1.)/(8.*Frsq*(2.+Frsq))
    return(jump_Eff*100.)

print(f"Jump Efficiency : {jump_Eff(6.920):.3f} percent")

Jump Efficiency : 38.391 percent


## Error Analysis

### Predicted vs. Observed Depth

Predicted sequent depth: 186 mm
Observed sequent depth: 190 mm

Absolute error (Observed − Predicted): $E=190−186=4 mm$

Percent error (relative to predicted): $\%E_{pred} = \frac{4}{186} \approx 2.2 \%$

**Likely Sources and Direction of Error:**

- Measurement location interpretation. The water surface near the hydraulic jump is irregular, and selecting the exact position for depth measurement can slightly alter results. A reading taken slightly upstream or downstream of the intended location could yield a few millimeters of difference.
- Standing wave effects. Surface oscillations can raise or lower the apparent water level depending on when the reading is taken. Measuring near the crest of a standing wave likely caused the observed depth to be slightly higher than the predicted value.

**How to Minimize Error**

- Take several quick readings at the same station and compute the average depth.
- Measure in a quieter region of the flow where surface fluctuations are smaller.
- Ensure the point gauge or digital probe is vertical and aligned with the same longitudinal station each time.

**Interpretation**

A difference of only 4 mm (≈2%) demonstrates excellent agreement between theoretical prediction and experimental observation.
Given the turbulent nature of the hydraulic jump and the difficulty of identifying a stable surface level, this result confirms that both the measurement technique and theoretical model are performing well.

### Predicted vs. Observed Length

Predicted jump length: 1156 mm
Observed jump length: 1200 mm

Absolute error (Observed − Predicted): $E=1200−1156=44 mm$

Percent error (relative to predicted): $\%E_{pred} = \frac{44}{1156} \approx 3.8 \%$

**Interpretation**
The predicted and observed jump lengths show excellent agreement, with less than 4% difference.
Considering that the observed value was derived from photo-based stationing rather than direct flume measurement, this level of correspondence suggests that the empirical relation used (Hager, 1992) provides a reasonable first-order estimate for jump extent under laboratory conditions.

In practice, the length of a hydraulic jump is not sharply defined. The flow transitions gradually from the highly turbulent roller region to a quasi-uniform downstream condition. As a result, determining where the jump “ends” often depends on the observer’s judgment or the chosen definition criterion—for example, where surface oscillations damp out or where mean velocity becomes approximately uniform.

For future experiments, the jump length should be measured systematically from the toe of the jump (identified visually or from depth data) to the end of the roller region where the surface becomes relatively smooth. Repeated measurements under steady conditions, ideally supported by video analysis or depth profiles, would allow for a more objective comparison with theoretical predictions.

Overall, the small discrepancy observed here reinforces the validity of the empirical formula and highlights that, in small-scale flumes, measurement uncertainty often rivals or exceeds modeling error. In field-scale structures, additional factors such as bed roughness, tailwater control, and three-dimensional effects would further influence jump length and energy dissipation.

:::{admonition} Key Takeaway(s)
- The **hydraulic jump length** scales primarily with the **upstream depth ($y_1$)** and **Froude number $(Fr_1$)**, reflecting the balance between incoming momentum and downstream resistance.  
- Agreement between predicted and observed lengths within a few percent confirms that the empirical relationship (Hager, 1992) captures the dominant physics of jump development.  
- In laboratory settings, **measurement uncertainty**—especially in locating the jump toe and end—often exceeds theoretical error; careful stationing and video review can improve precision.  
:::


## Summary and Conclusions

This experiment demonstrated the fundamental principles governing open-channel hydraulics and the formation of a hydraulic jump. Using a controlled flume setup, students determined discharge from a narrow-crested weir, created a stable hydraulic jump, and compared measured sequent depths and jump lengths with theoretical and empirical predictions.

The computed discharge of approximately **0.019 m³/s** established the flow conditions used to predict alternate and sequent depths. The predicted sequent depth (186 mm) and the observed value (190 mm) agreed within **2 percent**, indicating excellent consistency between theory and experiment. Similarly, the predicted and observed jump lengths (1156 mm and 1200 mm, respectively) differed by less than **4 percent**, reinforcing that empirical relationships such as those presented by Hager (1992) provide realistic estimates of hydraulic jump behavior in laboratory channels.

Energy and power loss calculations confirmed that a hydraulic jump is an effective energy dissipator. Even at small laboratory scale, a significant portion of the upstream specific energy was converted into turbulence and heat within the jump region. The measured head loss (≈ 0.32 m) and corresponding power dissipation emphasize the jump’s engineering role in protecting downstream structures from erosion and excessive velocities.

Discrepancies between predicted and observed quantities were primarily attributable to measurement uncertainty—especially in identifying the exact start and end of the jump—and to the unsteady nature of the roller. Improved measurement techniques, such as averaging multiple readings or analyzing video stills, would reduce these uncertainties in future trials.

Overall, the results validate theoretical models of hydraulic jumps and demonstrate how laboratory-scale experiments can effectively reproduce and analyze complex hydraulic phenomena. The exercise also illustrates key experimental principles—careful observation, repeatability, and comparison of theoretical and empirical methods—that are essential in professional engineering practice.

:::{admonition} Key Takeaway(s)
- The hydraulic jump provides a clear example of **momentum conservation** and **energy dissipation** in open-channel flow.  
- Agreement between measured and predicted values demonstrates that empirical design formulas can reliably estimate real-world behavior.  
- Accurate experimental results depend on **clear measurement definitions**, **repeatable procedures**, and **critical comparison** of data with theory.  
:::


## References

1. [Descriptive Statistics](http://54.243.252.9/engr-1330-webroot/engr1330notes/_build/html/examples/8statistical/1descriptivestatistics.html)
2. [Plotting Data](http://54.243.252.9/engr-1330-webroot/engr1330notes/_build/html/examples/7graphing/example05.html)
3. [Fitting Data Models](http://54.243.252.9/engr-1330-webroot/engr1330notes/_build/html/examples/10datamodels/datamodels.html)
4. [Holman, J.P., (2012) Experimental Methods for Engineers, 8th Ed. (Chapters 1-3)](https://mech.at.ua/HolmanICS.pdf)
5. [Holman, J.P., (2012) Experimental Methods for Engineers, 8th Ed. (Chapter 15 Report Writing)](https://mech.at.ua/HolmanICS.pdf)
6. [Hydraulic Jump Theory](https://en.wikipedia.org/wiki/Hydraulic_jump)
7. [Hydraulic Jump Calculator](https://www.omnicalculator.com/physics/hydraulic-jump)